In [4]:
import random

s = {5, 6, 7, 8}

random.sample(s, 2)

TypeError: Population must be a sequence.  For dicts or sets, use sorted(d).

In [5]:
trump_values = [0, 0, 14, 10, 20, 3, 4, 11]


def higher_trumps(trump, current_trick, hand_trumps):
    max_trick_trump_value = max(trump_values[card % 8] for card in current_trick if card // 8 == trump)
    return [card for card in hand_trumps if trump_values[card % 8] > max_trick_trump_value]

In [7]:
higher_trumps(0, [3, 23, 17], [*range(8)])

[2, 4, 7]

In [14]:
def get_scores(trump):
    scores = [0, 0, 0, 10, 2, 3, 4, 11] * 4
    scores[8 * trump : 8 * trump + 8] = [0, 0, 14, 10, 20, 3, 4, 11]
    return scores


def get_card_name(index):
    values = ["7", "8", "9", "10", "J", "Q", "K", "A"]
    suits = ["♠", "♥", "♦", "♣"]
    return values[index % 8] + suits[index // 8]


def get_ranks(trump, lead):
    ranks = [0] * 32
    ranks[8 * lead : 8 * lead + 8] = [0, 1, 2, 6, 3, 4, 5, 7]
    ranks[8 * trump : 8 * trump + 8] = [8, 9, 14, 12, 15, 10, 11, 13]
    return ranks


for i in range(32):
    print(f"{get_card_name(i)}: {get_scores(0)[i]:02} {get_ranks(0, 0)[i]}")

7♠: 00 8
8♠: 00 9
9♠: 14 14
10♠: 10 12
J♠: 20 15
Q♠: 03 10
K♠: 04 11
A♠: 11 13
7♥: 00 0
8♥: 00 0
9♥: 00 0
10♥: 10 0
J♥: 02 0
Q♥: 03 0
K♥: 04 0
A♥: 11 0
7♦: 00 0
8♦: 00 0
9♦: 00 0
10♦: 10 0
J♦: 02 0
Q♦: 03 0
K♦: 04 0
A♦: 11 0
7♣: 00 0
8♣: 00 0
9♣: 00 0
10♣: 10 0
J♣: 02 0
Q♣: 03 0
K♣: 04 0
A♣: 11 0


: 